In [53]:
import pandas as pd
import numpy as np

In [54]:
data_cols = ['user id','movie id','rating','timestamp']
data = pd.read_csv('/workspaces/LLM-Recommender-System/data/ml-100k/u.data', sep='\t', header=None, names=data_cols)
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [55]:
item_cols = ['movie id',  'movie title', 'release date', 'video release date','IMDb URL', 'unknown',\
'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama','Fantasy', \
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('/workspaces/LLM-Recommender-System/data/ml-100k/u.item',  sep='|',header=None,names=item_cols,encoding='latin-1')
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
# Assuming 'df' is your DataFrame
genre_columns = ['Action', 'Adventure', 'Animation', 'Children', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Function to get genres for a row
def get_genres(row):
    genres = [genre for genre in genre_columns if row[genre] == 1]
    return ', '.join(genres) if genres else 'unknown'

# Apply the function to each row
item['Genres'] = item.apply(get_genres, axis=1)
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"Animation, Children"
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,"Action, Adventure, Thriller"
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller


In [57]:
item = item[['movie id', 'movie title', 'Genres']]
item.head()

,movie id,movie title,Genres
0,1,Toy Story (1995),"Animation, Children"
1,2,GoldenEye (1995),"Action, Adventure, Thriller"
2,3,Four Rooms (1995),Thriller
3,4,Get Shorty (1995),Action
4,5,Copycat (1995),Thriller


In [58]:
# merge data and item
df = pd.merge(data, item, on='movie id').sort_values(by=['user id', 'movie id']).reset_index(drop=True)
df.head()

,user id,movie id,rating,timestamp,movie title,Genres
0,1,1,5,874965758,Toy Story (1995),"Animation, Children"
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller"
2,1,3,4,878542960,Four Rooms (1995),Thriller
3,1,4,3,876893119,Get Shorty (1995),Action
4,1,5,3,889751712,Copycat (1995),Thriller


In [59]:
# average rating for each movie for each user, add the column to df
df['avg_rating'] = df.groupby(['user id', 'movie id'])['rating'].transform('mean')
# rename the column
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp', 'movie_title', 'genres', 'avg_rating']
df.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,1,1,5,874965758,Toy Story (1995),"Animation, Children",5.0
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller",3.0
2,1,3,4,878542960,Four Rooms (1995),Thriller,4.0
3,1,4,3,876893119,Get Shorty (1995),Action,3.0
4,1,5,3,889751712,Copycat (1995),Thriller,3.0


In [60]:
df.to_csv('/workspaces/LLM-Recommender-System/data/processed_movie100k.csv', index=False)